In [ ]:
!pip install scipy dask matplotlib

In [ ]:
from pathlib import Path
from zipfile import ZipFile

from rich.progress import track
from jupytergis.tiler import GISDocument
import httpx
import rioxarray
import xarray as xr
import scipy.ndimage

We first download the awesome [HydroSHEDS](https://hydrosheds.org) dataset, and in particular the flow accumulation for South America. You can think of flow accumulation as a potential river flow, so we will have a visual representation of rivers, including the great Amazon river.

In [ ]:
url = "https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/sa_acc_30s_grid.zip"
filename = Path(url).name
name = filename[: filename.find("_grid")]
adffile = Path(name) / name / "w001001.adf"

if not adffile.exists():
    with httpx.stream("GET", url) as r, open(filename, "wb") as f:
        total = int(r.headers["Content-Length"]) / 1024
        for data in track(
            r.iter_bytes(chunk_size=1024), total=total, description="Downloading"
        ):
            f.write(data)
        f.flush()
    zip = ZipFile(filename)
    zip.extractall(".")

Let's open the box and see what's in there.

In [ ]:
da = rioxarray.open_rasterio(adffile, masked=True)
da

We just need to select the band. We will also chunk it in order to improve performances.

In [ ]:
da = da.sel(band=1)
da = da.chunk(dict(x=1000, y=1000))

In [ ]:
vmin, vmax = int(da.min().compute()), int(da.max().compute())
vmin, vmax

In [ ]:
from titiler.core.algorithm import BaseAlgorithm
from rio_tiler.models import ImageData
import numpy as np

class Log(BaseAlgorithm):
    def __call__(self, img: ImageData) -> ImageData:
        radius = 2
        circle = np.zeros((2*radius+1, 2*radius+1)).astype("uint8")
        y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
        index = x**2 + y**2 <= radius**2
        circle[index] = 1
        data = np.sqrt(img.data[0])
        data = scipy.ndimage.maximum_filter(data, footprint=circle)
        return ImageData(
            data,
            img.mask,
            assets=img.assets,
            crs=img.crs,
            bounds=img.bounds,
        )

In [ ]:
doc = GISDocument()
doc

In [ ]:
await doc.add_tiler_layer(
    name="Flow accumulation layer",
    data_array=da,
    colormap_name="viridis",
    rescale=np.sqrt([vmin, vmax]),
    algorithm=Log,
    reproject="max",
)